In [ ]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, WRITING_DPO_CONFIG, SMOL_LM_135M

cfg = WRITING_DPO_CONFIG
cfg.train_batch_size = 1
cfg.model_id_or_path = SMOL_LM_135M

In [ ]:
wrapper = TrainerWrapper(cfg, True)
wrapper.init_model()

In [ ]:
wrapper.init_data_module()

In [ ]:
wrapper.data_module.train_dataset

In [ ]:
from synthetic_data.utils import dictl
from IPython.display import display, Markdown
for i, sample in enumerate(wrapper.data_module.train_dataset.iter(1)):
    sample = dictl(sample)[0]
    display(Markdown(f"#### Scores: {sample['score_chosen']} / {sample['score_rejected']}"))
    display(Markdown(f"#### Models: {sample['model_chosen']} / {sample['model_rejected']}"))
    display(Markdown(f"### Instruction\n{sample['prompt']}"))
    display(Markdown(f"### Chosen\n{sample['chosen']}"))
    display(Markdown(f"### Rejected\n{sample['rejected']}"))
    scores_chosen, scores_rejected = sample["score_breakdown_chosen"], sample["score_breakdown_rejected"]
    score_text = ""
    for k, v in scores_chosen.items():
        if k in scores_rejected and v is not None and scores_rejected[k] is not None:
            score_text += f"\n{k}: {v} / {scores_rejected[k]}\n"
    display(Markdown(score_text))
    display(Markdown("---"))
    if i > 5:
        break

In [44]:
wrapper.init_trainer()

2025-04-06 23:37:15.854 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:373 - Saving output to: ./runs/04-06-23-37-860382-smollm2-135m-instruct--txt-bt-dpo
2025-04-06 23:37:15.857 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:396 - Initializing trainer, run_name: 04-06-23-37-860382-smollm2-135m-instruct--txt-bt-dpo, wandb project: gutenberg
2025-04-06 23:37:15.857 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:403 - logprobs cache location: ../dataset_caches/writing_d_p_o_data_module/b68b303b/ref_logprobs_cache peft config: False
Extracting prompt in train dataset:   0%|          | 0/9315 [00:00<?, ? examples/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Tokenizing eval dataset: 100%|██████████| 1035/1035 [0

In [ ]:
from model.reasoning import GSM8K_SYSTEM_PROMPT, CONNECTIONS_PROMPT
from model.utils import get_available_device

messages = [
    {"role": "system", "content": CONNECTIONS_PROMPT},
    {
        "role": "user",
        "content": "train, panda, dove, series, wind, bear, orca, bass, string, skunk, speed, sand, zebra, tourist, desert, chain",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_attention_mask=True,
)

device = get_available_device()
tokenized_chat = tokenized_chat.to(device)
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out, skip_special_tokens=True))

In [46]:
wrapper.train()

2025-04-06 23:40:37.207 | INFO     | trl_wrapper.trainer_wrapper:train:644 - Starting training.
2025-04-06 23:40:38.014 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:137 - Generating samples...
2025-04-06 23:40:38.014 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:59 - Generating policy samples, max length: 1024...
/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


KeyboardInterrupt: 